In [1]:
import pandas as pd
import difflib
import uuid

from datetime import datetime
from dateutil.relativedelta import relativedelta

from project_huishoudboekje.read_pdf import ReadPdf
from project_huishoudboekje.find_category import FindCategory

from project_huishoudboekje.config import GeneralSettings

In [2]:
file_name = 'NL45RABO0372430163-EUR-2022-02.pdf'

In [3]:
df = ReadPdf().run([file_name])

In [122]:
df['TRANS_ID'] = [uuid.uuid4() for _ in range(len(df.index))]

df = df.assign(
    SUB_ITEM = 1,
    SUB_CATEGORY = 'Niet-gecategoriseerd').set_index('TRANS_ID')

df['ANALYSE_IND'] = df['TRANSACTION_TYPE'].apply(lambda x: 0 if x == 'tb' else 1)

In [123]:
df.head()

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,SUB_ITEM,SUB_CATEGORY,ANALYSE_IND
TRANS_ID,,,,,,,,
c2c074f7-04f9-4229-b563-f077e652527b,2022-02-01,ei,debet,Nationale-Nederlanden Verzekeren INCASSO,17.27,1,Niet-gecategoriseerd,1
0eee4de3-af7c-44f0-84be-5854c4234453,2022-02-01,ei,debet,SPOTIFY SpotifyNL P198766322,9.99,1,Niet-gecategoriseerd,1
9ea74b3c-bd76-4d3e-a4be-df83df3f1403,2022-02-01,ei,debet,AEGON NEDERLAND NV,1261.03,1,Niet-gecategoriseerd,1
ba560c23-3087-4d96-a66b-31e695d15c94,2022-02-01,db,debet,Kosten,3.35,1,Niet-gecategoriseerd,1
0568592e-bfe2-4331-a6d8-9cd778ef6e13,2022-02-02,bc,debet,Kruidvat 3646 AMERSFOORT,8.77,1,Niet-gecategoriseerd,1


In [21]:
df_trans = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions.xlsx')

In [22]:
df_trans.head()

,TRANS_ID,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,SUB_ITEM,SUB_CATEGORY,ANALYSE_IND
0,cc01bfec-48d6-4a49-bac3-42ad4274c11e,2022-02-01,id,debet,Flatex Bank AG,2000.00,1,Beleggingen,1
1,d954a58d-3968-4bfc-b617-d412a38e2ffc,2022-01-31,cb,credit,Centrale Salarisadministratie Personeelsnummer:,19.00,1,Teruggave reiskosten,1
2,973c4a6f-b9e4-4439-9a04-8d151789e40f,2022-01-31,ei,debet,Basic Fit Nederland,21.66,1,Sportschool,1
3,5e57cc89-0a13-4ce8-8ac6-7e38855024c0,2022-01-31,id,debet,Airbnb B5CBXLWQB9T73VB2JFQF 01802030180511,1128.83,1,"Vakantie, weekend weg",1
4,14268353-e9e5-4c30-a1fc-aa305c600def,2022-01-31,id,debet,Gemeente Amersfoort 7543304,722.88,1,Gemeentelijke belastingen,1


In [124]:
list_parties = df_trans['PARTY'].tolist()

for idx in df.index:
    
    if df.loc[idx, 'SUB_CATEGORY'] == 'Niet-gecategoriseerd':
        party = df.loc[idx, 'PARTY']
        amount = df.loc[idx, 'AMOUNT']
        fin_type = df.loc[idx, 'FINANCIAL_TYPE']
        day = df.loc[idx, 'DATE'].day

        matches = difflib.get_close_matches(party, df_trans['PARTY'], n=3, cutoff=.6)
        print(matches)
        if matches:
            df_match = df_trans.loc[df_trans.PARTY.isin(matches), :]

            df, success = check_previous_month(df_match, df, idx, amount, fin_type)
            
            if not success:
                df = find_most_likely_category(df_match, df, idx, amount, fin_type)
            
            
    

['Nationale-Nederlanden Verzekeren INCASSO', 'Nationale-Nederlanden']
[]
['AEGON NEDERLAND NV']
['Kosten']
['Kruidvat 3646 AMERSFOORT', 'Kruidvat 3646 AMERSFOORT', 'Kruidvat 3646 AMERSFOORT']
['Hoogvliet 700 6 AMERSFOORT', 'Hoogvliet 700 6 AMERSFOORT', 'Hoogvliet 700 7 AMERSFOORT']
['Tango Amersfoort 1 AMERSFOORT', 'Tango Amersfoort 2 AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT']
['Spar City Monarch ©S-GRAVENHAGE']
['J.P. Schuitemaker Netflixieee']
['Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT']
['BORNEMAN A M']
['STICHTING COMPASSION NEDERLAND']
['Stichting Bootvluchteling via']
['NETFLIX INTERNATIONAL B.V.']
['KPN B.V. Factuur']
['A.M. Borneman Compassion', 'M.T. Borneman Videoland']
['Kruidvat 3646 AMERSFOORT', 'Kruidvat 3646 AMERSFOORT', 'Kruidvat 3646 AMERSFOORT']
['Hoogvliet 700 5 AMERSFOORT', 'Hoogvliet 700 7 AMERSFOORT', 'Hoogvliet 700 6 AMERSFOORT']
['KPN - Mobiel']
['Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoor

['ANWB B.V. ARNL3335650870']
['IKEA Amersfoort AMERSFOORT', 'IKEA Amersfoort AMERSFOORT', 'Tango Amersfoort 2 AMERSFOORT']
['Centrale Salarisadministratie Personeelsnummer:', 'Centrale Salarisadministratie Personeelsnummer:', 'Centrale Salarisadministratie']
[]
['M.T. Borneman Videoland', 'A.M. Borneman Compassion']
['BELASTINGDIENST MAAND JAN.']
['VIDEOLAND DOOR BUCKAROO']
['Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT']
['Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT']
[]
[]
['Tango Amersfoort 2 AMERSFOORT', 'Tango Amersfoort 1 AMERSFOORT', 'Jumbo Amersfoort Wie AMERSFOORT']
[]
[]
[]
[]
[]
[]
['Centrale Salarisadministratie Personeelsnummer:', 'Centrale Salarisadministratie Personeelsnummer:', 'Centrale Salarisadministratie']
['Centrale Salarisadministratie BETALINGSPERIODE', 'Centrale Salarisadministratie', 'Centrale Salarisadministratie Personeelsnummer:']
['Hoogvliet 70

In [104]:
def find_most_likely_category(df_match, df, idx, amount, fin_type):
    
    df_sel = df_match.loc[df_match.FINANCIAL_TYPE == fin_type]
    
    most_frequent_cat = df_sel.SUB_CATEGORY.value_counts().index[0]
    
    df.loc[idx, 'SUB_CATEGORY'] = most_frequent_cat
    
    return df

In [114]:
def check_previous_month(df, df_new, idx, amount, fin_type):
    
    previous_date = df_new.loc[idx, 'DATE'] - relativedelta(months=1)

    df_sel = df.loc[(df.DATE.dt.year == previous_date.year) & (df.DATE.dt.month == previous_date.month)]
    
    date_sel = df_sel['DATE'].iloc[0].day
    
    if df_sel.AMOUNT.sum() == amount and df_sel['FINANCIAL_TYPE'].iloc[0] == fin_type and \
        date_sel >= max(previous_date.day - 4, 1) and date_sel <= max(previous_date.day + 4, 31):
        list_cats = df_sel[['AMOUNT', 'SUB_ITEM', 'SUB_CATEGORY']].values.tolist()
        
        if len(list_cats) > 1:
            df_new = df_new.append([df_new.loc[idx]]*(len(list_cats)-1))

            df_new.loc[idx, ['AMOUNT', 'SUB_ITEM', 'SUB_CATEGORY']] = list_cats
        else:
            df_new.loc[idx, 'SUB_CATEGORY'] = list_cats[0][2]
            
        success=True
    else:
        success=False
    
    return df_new, success


In [81]:
test = [[1261.03, 1, 'Hypotheek']]

In [82]:
test[0][2]

'Hypotheek'

In [125]:
df.to_excel(GeneralSettings.project_path / 'data/test_maand2_nw.xlsx', index=True)

In [4]:
df2 = FindCategory().run(df)

In [5]:
df2

,DATE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,SUB_ITEM,SUB_CATEGORY,ANALYSE_IND
TRANS_ID,,,,,,,,
54c3b035-35c0-4f9e-83d7-839ce9f13278,2022-02-01,ei,debet,Nationale-Nederlanden Verzekeren INCASSO,10.94,1,Opstalverzekering,1
9d980667-2872-4088-a352-59b1bfe8625e,2022-02-01,ei,debet,SPOTIFY SpotifyNL P198766322,9.99,1,Niet-gecategoriseerd,1
2f2fd33e-06ec-4612-bed6-6a88e8d4822f,2022-02-01,ei,debet,AEGON NEDERLAND NV,1261.03,1,Hypotheek,1
231ea26a-7646-45eb-95ff-b85d17327ed7,2022-02-01,db,debet,Kosten,3.35,1,Betaalpakket,1
49733baa-3571-4094-a7b5-a82e2aaf5244,2022-02-02,bc,debet,Kruidvat 3646 AMERSFOORT,8.77,1,Boodschappen (non-food),1
...,...,...,...,...,...,...,...,...
1e411734-92b4-4eeb-bbeb-bef99e8ee330,2022-02-28,cc,debet,Rabobank Nederland,92.73,1,Amazon Prime,1
79d2bd1f-f575-4ade-9d54-4e8531b80cb1,2022-02-28,ei,debet,BELASTINGDIENST 89-XZR-3 28-01-2022,33.00,1,Wegenbelasting,1
939973ee-f6bf-4791-b538-79296a759e89,2022-02-28,bv,credit,Betaalverzoek Rabobank NL61ASNB0267337108:AM,44.63,1,Niet-gecategoriseerd,1
